# Working with HEALPix data

[HEALPix](https://healpix.jpl.nasa.gov/) (Hierarchical Equal Area isoLatitude Pixelisation) is an algorithm that is often used to store data from all-sky surveys.

There are several tools in the Astropy ecosystem for working with HEALPix data, depending on what you need to do:

* The [astropy-healpix](https://astropy-healpix.readthedocs.io/en/latest/index.html) coordinated package is a BSD-licensed implementation of HEALPix which focuses on being able to convert celestial coordinates to HEALPix indices and vice-versa, as well as providing a few other low-level functions.

* The [reproject](https://reproject.readthedocs.io/en/stable/) coordinated package (which we've already looked at) includes functions for converting from/to HEALPix maps.

* The [HiPS](https://hips.readthedocs.io/en/latest/) affiliated package implements suport for the [HiPS](http://aladin.u-strasbg.fr/hips/) scheme for storing data that is based on HEALPix.

In this tutorial, we will take a look at the two first one of these, but we encourage you to learn more about HiPS too!


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Convert between celestial coordinates and HEALPix indices</li>
<li>Find the boundaries of HEALPix pixels</li>
<li>Find healpix pixels close to a position</li>
<li>Reproject a HEALPix map to a standard projection</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy-healpix and reproject. For more information about the features presented below as well as other available features, you can read the
[astropy-healpix](https://astropy-healpix.readthedocs.io/en/latest/index.html) and the [reproject](https://reproject.readthedocs.io/en/stable/) documentation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Data

For this tutorial, we will be using a downsampled version of the Planck HFI 857Ghz map which is stored as a HEALPix map ([data/HFI_SkyMap_857_2048_R1.10_nominal_ZodiCorrected_lowres.fits](data/HFI_SkyMap_857_2048_R1.10_nominal_ZodiCorrected_lowres.fits)).

## Using astropy-healpix

To start off, we can open the HEALPix file (which is a FITS file) with astropy.io.fits:

In [ ]:
from astropy.io import fits
hdulist = fits.open('data/HFI_SkyMap_857_2048_R1.10_nominal_ZodiCorrected_lowres.fits')
hdulist.info()

The HEALPix map values are stored in HDU 1. This HDU also contains useful header information that helps us understand how to interpret the HEALPix values:

In [ ]:
hdulist[1].header['NSIDE']

In [ ]:
hdulist[1].header['ORDERING']

In [ ]:
hdulist[1].header['COORDSYS']

With this information we can now construct a ``HEALPix`` object:

In [ ]:
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic

In [ ]:
hp = HEALPix(nside=hdulist[1].header['NSIDE'],
             order=hdulist[1].header['ORDERING'],
             frame=Galactic())  

We can then use this object to manipulate the HEALPix map. To start off, we can find out what the coordinates of specific pixels are:

In [ ]:
hp.healpix_to_skycoord([13322, 2231, 66432])

and vice-versa:

In [ ]:
from astropy.coordinates import SkyCoord
hp.skycoord_to_healpix(SkyCoord.from_name('M31'))

You can also find out what the boundaries of a pixel are:

In [ ]:
edge = hp.boundaries_skycoord(649476, step=100)
edge

The ``step`` argument controls how many points to sample along the edge of the pixel. The result should be a polygon:

In [ ]:
plt.plot(edge[0].l.deg, edge[0].b.deg)

You can find all HEALPix pixels within a certain radius of a known position:

In [ ]:
from astropy import units as u
hp.cone_search_skycoord(SkyCoord.from_name('M31'), radius=1 * u.deg)

And finally you can interpolate the map at specific coordinates:

In [ ]:
hp.interpolate_bilinear_skycoord(SkyCoord.from_name('M31'), hdulist[1].data['I_STOKES'])


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Find the mean value of I_STOKES within 2 degrees of M42</li>
<li>Use astropy.coordinates to check that all the pixels returned by the cone search are indeed within 2 degrees of M42 (if not, why not? Hint: check the documentation of <a href="https://astropy-healpix.readthedocs.io/en/latest/api/astropy_healpix.HEALPix.html#astropy_healpix.HEALPix.cone_search_skycoord">cone_search_skycoord()</a>)</li>
</ol>

</div>

</section>


In [ ]:
#1
import numpy as np
M42 = SkyCoord.from_name('M42')
m42_pixels = hp.cone_search_skycoord(M42, radius=2 * u.deg)
print(np.mean(hdulist[1].data['I_STOKES'][m42_pixels]))

In [ ]:
#2
m42_cone_search_coords = hp.healpix_to_skycoord(m42_pixels)
separation = m42_cone_search_coords.separation(M42).degree
_ = plt.hist(separation, bins=50)

## Using reproject for HEALPix data

The reproject package is useful for HEALPix data to convert a HEALPix map to a regular projection, and vice-versa. For example, let's define a simple all-sky Plate-Caree WCS:

In [ ]:
from astropy.wcs import WCS
wcs = WCS(naxis=2)
wcs.wcs.ctype = 'GLON-CAR', 'GLAT-CAR'
wcs.wcs.crval = 0, 0
wcs.wcs.crpix = 180.5, 90.5
wcs.wcs.cdelt = -1, 1

We can now use [reproject_from_healpix](https://reproject.readthedocs.io/en/stable/api/reproject.reproject_from_healpix.html#reproject.reproject_from_healpix) to convert the HEALPix map to this header:

In [ ]:
from reproject import reproject_from_healpix

In [ ]:
array, footprint = reproject_from_healpix('data/HFI_SkyMap_857_2048_R1.10_nominal_ZodiCorrected_lowres.fits',
                                          wcs, shape_out=(180, 360))

In [ ]:
plt.imshow(array, vmax=100)

You can also use [reproject_to_healpix](https://reproject.readthedocs.io/en/stable/api/reproject.reproject_to_healpix.html#reproject.reproject_to_healpix) to convert a regular map to a HEALPix array.


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Reproject the HFI HEALPix map to the projection of the GAIA point source density map as well as the IRAS map that we used in previous tutorials.</li>
<li>Visualize the results using WCSAxes and optionally the image normalization options.</li>
</ol>

</div>

</section>


In [ ]:
#1
header_gaia = fits.getheader('data/LMCDensFits1k.fits')
header_irsa = fits.getheader('data/ISSA_100_LMC.fits')

array_gaia, _ = reproject_from_healpix('data/HFI_SkyMap_857_2048_R1.10_nominal_ZodiCorrected_lowres.fits',
                                       header_gaia)
array_irsa, _ = reproject_from_healpix('data/HFI_SkyMap_857_2048_R1.10_nominal_ZodiCorrected_lowres.fits',
                                       header_irsa)

In [ ]:
#2
from astropy.visualization import simple_norm
ax = plt.subplot(projection=WCS(header_gaia))
im =ax.imshow(array_gaia, cmap='plasma',
              norm=simple_norm(array_gaia, stretch='sqrt', percent=99.5))
plt.colorbar(im)
ax.grid()
ax.set_xlabel('Galactic Longitude')
ax.set_ylabel('Galactic Latitude')

<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)